In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext,SparkConf
spark = SparkSession.builder.\
            master("local").\
            appName("my App Name").\
            getOrCreate()
sc = spark.sparkContext

from pyspark.ml.feature import *
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline,PipelineModel

from pyspark.ml.linalg import Vector
from pyspark.sql import Row
training = spark.createDataFrame([(0,"a b c d e spark",1.0,),\
                                (1,"b d",0.0,),(2,"spark f g h ",1.0,),\
                                (3, "hadoop mapreduct",0.0,)],["id","text","label"])
training.show()
tokenizer = Tokenizer(inputCol="text",outputCol="words",)
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(),outputCol="features",numFeatures=1000)
lr = LogisticRegression().setMaxIter(10).setRegParam(0.01)
pipeline = Pipeline(stages=[tokenizer,hashingTF,lr])
model = pipeline.fit(training)
test = spark.createDataFrame([(4,"spark i j k"),(5,"l m n"),(6,"spark a"),(7,"apache hadoop")],["id","text"])
model.transform(test).show()

+---+----------------+-----+
| id|            text|label|
+---+----------------+-----+
|  0| a b c d e spark|  1.0|
|  1|             b d|  0.0|
|  2|    spark f g h |  1.0|
|  3|hadoop mapreduct|  0.0|
+---+----------------+-----+

+---+-------------+----------------+--------------------+--------------------+--------------------+----------+
| id|         text|           words|            features|       rawPrediction|         probability|prediction|
+---+-------------+----------------+--------------------+--------------------+--------------------+----------+
|  4|  spark i j k|[spark, i, j, k]|(1000,[105,149,32...|[0.16293291377589...|[0.54064335448523...|       0.0|
|  5|        l m n|       [l, m, n]|(1000,[6,638,655]...|[2.64074492868042...|[0.93343826273835...|       0.0|
|  6|      spark a|      [spark, a]|(1000,[105,170],[...|[-1.7313553283508...|[0.15041430048073...|       1.0|
|  7|apache hadoop|[apache, hadoop]|(1000,[181,495],[...|[3.74294051364969...|[0.97686361395183...|  